In [1]:
# implemenatation of https://arxiv.org/pdf/1703.07754.pdf

# To do 

# create basic model bidirectional architecture [ Done ]
# glove initialization [ Done ]
# ctc integration 
# dataset preparation [ Done ]
# feature extraction [ Done ]



In [22]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import logfbank
from tensorflow.contrib.data import Dataset, Iterator


In [63]:
'''We extracted 40-dimensional logMel filterbank energies over
25 ms frames every 10 ms from the input speech signal, stacked
two successive frames, and dropped every alternate frame resulting
in 80-dimensional logMel features at half the rate of the
original 40-dimensional features. This ”stacking+decimation”
operation [16] provided significant speed-up in training because
the sequence length reduces by half with no loss in performance '''


def audio_to_features(fileurl):
    rate, sig = wav.read(fileurl)
  #  mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
    fbank_feat = logfbank(sig,rate,nfilt=40)
    

    
    # if length of frames is even no problemo, otherwise add a new frame of zeroes
    
    if fbank_feat.shape[0]%2 == 0:
        print('even')
        fbank_feat = np.reshape(fbank_feat, (fbank_feat.shape[0]/2,80))
        return fbank_feat
    else:
        print('odd')
        print(fbank_feat.shape)
        
        zero_array=np.zeros(40)
      #  print(zero_array)
       
        fbank_feat =          np.vstack((fbank_feat , zero_array))
        #print(fbank_feat.shape)
        
        new_length=fbank_feat.shape[0]//2
        
        fbank_feat = np.reshape(fbank_feat, ( new_length ,80))
        return fbank_feat

    

In [64]:
audio_to_features('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav').shape

odd
(299, 40)
(300, 40)


(150, 80)

In [3]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
    
       
       

       return parsed_line[0] , parsed_line[1]

In [4]:
def _read_py_function(audio, label):
    audio =audio_to_features(audio)
   
    return audio ,label

In [5]:
dataset = tf.contrib.data.TextLineDataset("/home/saurabh/Documents/tf_orange/tf_orange/data/data.csv")
dataset=dataset.map(decode_csv)



dataset = dataset.map(
    lambda audio, label: tuple(tf.py_func(
        _read_py_function, [audio, label], [tf.double, label.dtype])))




dataset=dataset.batch(2)

In [3]:
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 2
display_step = 10

# Network Parameters
vocab_size=100 # this means number of words at output layer, CTC blank symbol exluded
num_input = 80# 
timesteps = 300 # timesteps
num_hidden = 320 # hidden layer num of features


In [ ]:
# method to load pretrained gloVe model

def loadGloVe(filename):
    vocab = []
    embd = []
    file = open(filename,'r')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append(row[1:])
    print('Loaded GloVe!')
    file.close()
    return vocab,embd

#vocab,embd = loadGloVe(filename)
#vocab_size = len(vocab)
#embedding_dim = len(embd[0])
#embedding = np.asarray(embd)

In [4]:
# Define weights

#W = tf.Variable(tf.constant(0.0, shape=[vocab_size, embedding_dim]),
 #               trainable=False, name="W")

# Initialize weights as glove vectors    

weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.constant(0.0, shape=[embedding_dim,vocab_size ]),
                trainable=False, name="W")
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size + 1]))
}


embedding_placeholder = tf.placeholder(tf.float32, [vocab_size, embedding_dim])
embedding_init = weights['out'].assign(embedding_placeholder)

In [21]:
a = np.random.rand(4, 13)
print(a.shape)

b = np.reshape(a, (2,26))

print(b.shape)



#for (x,y), value in numpy.ndenumerate(a):
#...  print x,y

(4, 13)
(2, 26)


In [5]:



X = tf.placeholder("float", [None, timesteps, num_input])

In [6]:
def model(x):

   
    #  data input shape: (batch_size, timesteps, n_input)
    

    # Define lstm cells with tensorflow
    # Forward direction cell for layer 1
    lstm_fw_cell_1 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell for layer 1 
    lstm_bw_cell_1 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
     # Forward direction cell for layer 2
    lstm_fw_cell_2 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell for layer 2 
    lstm_bw_cell_2 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    # Forward direction cell for layer 3
    lstm_fw_cell_3 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell for layer 3 
    lstm_bw_cell_3 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    # Forward direction cell for layer 4
    lstm_fw_cell_4 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell for layer 4 
    lstm_bw_cell_4 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
     # Forward direction cell for layer 5
    lstm_fw_cell_5 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell for layer 5 
    lstm_bw_cell_5 = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    cells_fw=[lstm_fw_cell_1,lstm_fw_cell_2,lstm_fw_cell_3,lstm_fw_cell_4,lstm_fw_cell_5]
    cells_bw=[lstm_bw_cell_1, lstm_bw_cell_2,lstm_bw_cell_3,lstm_bw_cell_4,lstm_bw_cell_5]

    outputs, _, _ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, x,
                                              dtype=tf.float32)
    # final linear dense layer
    return tf.matmul(outputs, weights['out']) + biases['out']
    #return outputs

In [7]:
logits = model(X)

In [8]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [9]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    input_features=np.random.rand(2,300,80)
    print(sess.run(logits, feed_dict={X: input_features}).shape)

(2, 300, 101)
